In [3]:
from flash_attn import flash_attn_func
import torch

batch_size = 4        # 4 örnek aynı anda
seqlen = 128          # her örnek 128 token uzunluğunda
hidden_dim = 768      # modelin gömme (embedding) boyutu
head_dim = 64         # her attention head'in boyutu

q = torch.randn(4, 128, 12, 64).half().cuda()
k = torch.randn(4, 128, 4, 64).half().cuda()
v = torch.randn(4, 128, 4, 64).half().cuda()


flash_attn_func(q, k, v)

tensor([[[[-3.3228e-01, -1.4502e-01,  6.7024e-03,  ...,  1.1981e-01,
           -1.0455e-01,  2.2034e-02],
          [ 5.0187e-05,  1.6821e-01, -4.9774e-02,  ...,  2.2961e-01,
           -2.8717e-02, -7.9956e-02],
          [-1.1438e-01,  3.8849e-02, -2.3193e-01,  ..., -1.0394e-01,
           -1.7676e-01, -4.1687e-02],
          ...,
          [-1.6833e-01, -1.3208e-01, -1.9440e-02,  ...,  1.9922e-01,
            1.8945e-01,  1.9568e-01],
          [-6.5430e-02, -9.6680e-02, -1.0056e-02,  ...,  1.2378e-01,
            7.0923e-02,  2.6904e-01],
          [ 7.2266e-02,  3.8853e-03,  1.5884e-02,  ...,  1.2329e-01,
            2.0935e-01,  2.8351e-02]],

         [[-1.1462e-01,  1.0510e-01,  1.4880e-01,  ...,  1.0022e-01,
           -1.0901e-01,  3.3264e-02],
          [-2.9639e-01,  1.5857e-01, -2.5977e-01,  ...,  8.1543e-02,
            3.0945e-02, -6.9962e-03],
          [-1.4270e-01,  1.3147e-01, -3.2990e-02,  ...,  2.8336e-02,
           -5.8228e-02,  1.0651e-01],
          ...,
     

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from flash_attn.modules.mha import MHA  # FlashAttention MHA
import random
torch.manual_seed(42)
random.seed(42)

class FlashAttentionBlock(nn.Module):
    def __init__(self, hidden_dim=768, n_heads=12, dropout=0.0):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads
        
        # LayerNorm (isteğe bağlı)
        self.ln = nn.LayerNorm(hidden_dim)
        
        # FlashAttention MHA; bu modül kendi içinde QKV projeksiyonu yapar.
        self.attn = MHA(
            embed_dim=hidden_dim,
            num_heads=n_heads,
            dropout=dropout,
            causal=True,
            device="cuda",  # ya "cuda" ya da "cpu"
            dtype=torch.float16,  # FlashAttention için FP16 tercih edilir.
            use_flash_attn=True
        )
        
        # Gerekirse, çıktı üzerinde ek bir lineer projeksiyon ekleyebilirsin.
        # Burada çıktı, (total, n_heads, head_dim) şeklinde olacak.
        # Çıkışı hidden_dim'e (yani n_heads * head_dim) birleştirmek için:
        self.out_proj = nn.Linear(hidden_dim, hidden_dim).to(dtype=torch.float16)

    def forward(self, x, attention_mask):
        """
        x: (B, T, hidden_dim)
        attention_mask: (B, T) boolean tensor; 1 gerçek token, 0 pad
        """
        B, T, C = x.size()
        # Önce layer norm
        x = self.ln(x)
        
        # FlashAttention varlen formatı kullanacaksak, giriş x'i (total, hidden_dim) haline getirmeliyiz.
        # Burada total, batch içindeki toplam gerçek token sayısıdır.
        # Eğer padding varsa, attention_mask'ten token sayısını alıp cu_seqlens oluşturacağız.
        seq_lens = attention_mask.sum(dim=1)  # her örnekteki gerçek token sayıları, shape: (B,)
        cu_seqlens = F.pad(seq_lens.cumsum(0), (1, 0), value=0).to(torch.int32)  # shape: (B+1,)
        max_seqlen = seq_lens.max().item()  # batch içindeki en uzun dizi
        
        # x'i "packed" formata getir: (total, hidden_dim)
        x_flat = x.reshape(B * T, C).contiguous()  # Not: Eğer padding varsa, bu tüm T kullanır.
        # Not: Gerçekten istenen, yalnızca gerçek tokenlar (attention_mask==1) olabilir.
        # Ancak, FlashAttention cu_seqlens kullanırken T'nin tüm elemanlarını (padding dahil) alır;
        # cu_seqlens, her örnekteki gerçek token sayısını içerir.
        
        # FlashAttention çağrısı:
        # self.attn; beklediği input x: (total, hidden_dim)
        # Çıkış shape: (total, n_heads, head_dim)
        out = self.attn(x_flat, cu_seqlens=cu_seqlens, max_seqlen=max_seqlen)
        
        # İsteğe bağlı: Çıkışı birleştirip, çıkış projesi uygulamak.
        # Önce, (total, n_heads, head_dim) → (total, hidden_dim)
        out = out.view(B * T, C)
        out = self.out_proj(out)
        
        # Son olarak, tekrar (B, T, hidden_dim) haline getir:
        out = out.view(B, T, C)
        return out

# Parametreler:
B, T, C = 5, 10, 768  # Batch size, sequence length, hidden_dim

# Dummy giriş (random tensor), FP16 olarak üret
x = torch.randn(B, T, C).cuda().half()

# Dummy attention mask: her batch için örnek
# İlk batch: 9 gerçek token, 3 padding; ikinci batch: tamamen dolu (12 gerçek token)
attention_mask = torch.tensor([
    [1]*9 + [0]*3,
    [1]*12
], dtype=torch.bool).cuda()

# Modeli oluştur ve GPU'ya al, FP16 kullan (ağırlıklar da FP16 olacak)
model = FlashAttentionBlock(hidden_dim=C, n_heads=12, dropout=0.0).cuda().half()

# Test et
output = model(x, attention_mask)
print("Output shape:", output.shape)  # Beklenen: (B, T, C)



Output shape: torch.Size([5, 10, 768])


In [6]:
# Reasoning Instruction
SYSTEM_PROMPT = """
A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <thinking> </thinking> and
<answer> </answer> tags, respectively, i.e., <thinking> reasoning process here </thinking><answer> answer here </answer>.
Response Format rules:
- Always start your response with <thinking> tag and end with </answer>.
- Do not include any text or commentary before the opening <thinking> tag or after the closing </answer> tag.
- Do not include any text or commentary between the closing </thinking> tag and the opening <answer> tag.
For example, your response follow this format:
<thinking>
[Your detailed chain-of-thought goes here]
</thinking>
<answer>
[Your final answer goes here]
</answer>
"""

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "How to add two numbers in Python?\n"},
]
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=256, temperature=0.2, top_p=0.9, do_sample=True, use_cache=False)
print(tokenizer.decode(outputs[0]))


model.safetensors:  61%|######    | 2.09G/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

<|im_start|>system

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <thinking> </thinking> and
<answer> </answer> tags, respectively, i.e., <thinking> reasoning process here </thinking><answer> answer here </answer>.
Response Format rules:
- Always start your response with <thinking> tag and end with </answer>.
- Do not include any text or commentary before the opening <thinking> tag or after the closing </answer> tag.
- Do not include any text or commentary between the closing </thinking> tag and the opening <answer> tag.
For example, your response follow this format:
<thinking>
[Your detailed chain-of-thought goes here]
</thinking>
<answer>
[Your final answer goes here]
</answer>
<|im_end|>
<|im_start|>user
How to add two numbers in Python?
<|im_end|>
<|im_start|>assistant


In [14]:
import re
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_cosine_schedule_with_warmup

# Reasoning Instruction
SYSTEM_PROMPT = """
A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <thinking> </thinking> and
<answer> </answer> tags, respectively, i.e., <thinking> reasoning process here </thinking><answer> answer here </answer>.
Response Format rules:
- Always start your response with <thinking> tag and end with </answer>.
- Do not include any text or commentary before the opening <thinking> tag or after the closing </answer> tag.
- Do not include any text or commentary between the closing </thinking> tag and the opening <answer> tag.
For example, your response follow this format:
<thinking>
[Your detailed chain-of-thought goes here]
</thinking>
<answer>
[Your final answer goes here]
</answer>
"""

# Helpers

def get_user_prompt(prompt: str) -> str:
    match = re.search(r"<\|im_start\|>user\s*(.*?)\s*<\|im_end\|>", prompt, re.DOTALL)
    if match:
        return match.group(1).strip()
    lines = prompt.splitlines()
    result = []
    for line in lines:
        if not line.strip().lower().startswith("system"):
            if line.strip().lower().startswith("user"):
                result.append(line.strip()[4:].strip())
            else:
                result.append(line)
    return "\n".join(result).strip()

def get_assistant_response(text: str) -> str:
    match = re.search(r"<\|im_start\|>assistant\s*(.*?)\s*<\|im_end\|>", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    lines = text.splitlines()
    result = []
    capture = False
    for line in lines:
        stripped = line.strip()
        if stripped.lower().startswith("assistant"):
            capture = True
            continue
        if capture:
            result.append(line)
    return "\n".join(result).strip()

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str:
    if "####" not in text:
        return text.strip()
    return text.split("####", 1)[1].strip()

def count_xml(text: str) -> float:
    count = 0.0
    if text.count("<thinking>\n") == 1:
        count += 0.225
    if text.count("\n</thinking>\n") == 1:
        count += 0.225
    if text.count("\n<answer>\n") == 1:
        count += 0.225
        count -= len(text.split("\n</answer>")[-1]) * 0.001
    if text.count("\n</answer>\n") == 1:
        count += 0.225
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

def inference(prompt: str, model_path: str) -> str:
    device = "cuda"
    model_infer = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    tokenizer_infer = AutoTokenizer.from_pretrained(model_path)
    inputs = tokenizer_infer(prompt, return_tensors="pt", max_length=256, truncation=False)
    outputs = model_infer.generate(
        inputs["input_ids"].to(device),
        attention_mask=inputs["attention_mask"].to(device),
        max_new_tokens=256,
        
        pad_token_id=tokenizer_infer.eos_token_id,
        temperature=0.2,
        num_return_sequences=1,
        top_p=0.9, do_sample=True, use_cache=False
    )
    full_text = tokenizer_infer.decode(outputs[0])
    user_question = get_user_prompt(prompt)
    assistant_response = get_assistant_response(full_text)
    extracted_answer = extract_xml_answer(assistant_response)
    return f"{'='*10} Inference {'='*10}\nQuestion:\n{user_question}\n\nModel Response:\n{assistant_response}\n\nExtracted:\n{extracted_answer}\n{'='*12} End {'='*12}\n"


inference("How to add two numbers in Python?\n", "HuggingFaceTB/SmolLM2-1.7B-Instruct")

'========== Inference ==========\nQuestion:\nHow to add two numbers in Python?\n\nModel Response:\n\n\nExtracted:\n\n============ End ============\n'

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-15 17:54:57.311983: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 17:54:57.322513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739631297.333230  351737 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739631297.336533  351737 cuda_blas.cc

In [5]:
# 1. Model ve Tokenizer'ı Yükle
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"  # Mistral 7B modeli


In [12]:


device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "system", "content": "Senin adın Mia"},{"role": "user", "content": "Adın ne?"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=512, temperature=0.2, top_p=0.9, do_sample=True)

outputs

<|im_start|>system
Senin adın Mia<|im_end|>
<|im_start|>user
Adın ne?<|im_end|>



tensor([[    1,  9690,   198, 27025,   254,   493, 20335,    94, 21230,     2,
           198,     1,  4093,   198,  4503, 20335,    94,   420,    47,     2,
           198,     1,   520,  9531,   198,  4503, 20335,    94,   420,    47,
             2]], device='cuda:0')

In [13]:
print(tokenizer.decode(outputs[0]))

<|im_start|>system
Senin adın Mia<|im_end|>
<|im_start|>user
Adın ne?<|im_end|>
<|im_start|>assistant
Adın ne?<|im_end|>
